In [1]:
! pip install google-cloud-bigquery pandas

In [2]:
%load_ext google.cloud.bigquery
%config InlineBackend.figure_format = "retina"

Idées:
- Calculer le temps moyen avant un ré-achat > tracer la courbe de probabilité moyen jour après jour de réachat
- Créer un petit algo simple qui prédit en fonction de la date de jour, date de premier achat, valeur du premier achat, type de premier achat, genre, country la probabilité d'acheter la semaine d'après > permet de sortir le top score par pays
- Faire un algo de reco (basée sur la matrice précédente/succession) du produit à mettre en avant pour augmenter proba de réacheter

In [3]:
%%bigquery
CREATE TABLE IF NOT EXISTS richemont.orders AS (
    SELECT
        client_id,
        DATE(transaction_date) AS transaction_at,
        SUM(price_full) AS total_price,
        COUNT(product_collection) AS quantity,
    FROM richemont.orders_raw
    GROUP BY 1, 2
)

Query is running:   0%|          |

""


In [4]:
%%bigquery
SELECT DISTINCT country_of_purchase FROM richemont.orders_raw

Query is running:   0%|          |

Downloading:   0%|          |

,country_of_purchase
0,China
1,France
2,USA
3,Switzerland
4,Japan
5,"Macau SAR, China"
6,"Hong Kong SAR, China"
7,South Korea
8,United Kingdom
9,"Russian Fed,"


In [7]:
%%bigquery
CREATE TABLE IF NOT EXISTS richemont.orders_region AS (
    SELECT
        CASE country_of_purchase
        WHEN "China" THEN "Asia Pacific"
        WHEN "France" THEN "Europe"
        WHEN "USA" THEN "Americas"
        WHEN "Switzerland" THEN "Europe"
        WHEN "Japan" THEN "Asia Pacific"
        WHEN "Macau SAR, China" THEN "Asia Pacific"
        WHEN "Hong Kong SAR, China" THEN "Asia Pacific"
        WHEN "South Korea" THEN "Asia Pacific"
        WHEN "United Kingdom" THEN "Europe"
        WHEN "Russian Fed," THEN "Europe"
        WHEN "Thailand" THEN "Asia Pacific"
        WHEN "Italy" THEN "Europe"
        WHEN "Utd,Arab Emir," THEN "Middle East & Africa"
        WHEN "Taiwan, China" THEN "Asia Pacific"
        WHEN "Brazil" THEN "Americas"
        WHEN "Malaysia" THEN "Asia Pacific"
        WHEN "Singapore" THEN "Asia Pacific"
        WHEN "Germany" THEN "Europe"
        WHEN "Spain" THEN "Europe"
        WHEN "Mexico" THEN "Americas"
        WHEN "Turkey" THEN "Middle East & Africa"
        WHEN "Canada" THEN "Americas"
        ELSE "Other"
        END AS region_of_purchase,
        country_of_purchase,
        CAST(SUM(price_full) AS INT) AS total_price,
    FROM richemont.orders_raw
    GROUP BY 1, 2
 )


Query is running:   0%|          |

""


In [7]:
%%bigquery
CREATE OR REPLACE TABLE `richemont.rfm` AS (
WITH
max_date AS (
  SELECT MAX(transaction_at) AS max_date_
  FROM richemont.orders
)

SELECT

  client_id,
  COUNT(DISTINCT DATE(transaction_at)) - 1 as frequency,
  DATE_DIFF(MAX(transaction_at), MIN(transaction_at), DAY) as recency,
  AVG(total_price) as monetary_value,
  DATE_DIFF(max_date.max_date_, MIN(transaction_at), DAY) as T

FROM richemont.orders, max_date

GROUP BY client_id, max_date_
)

Query is running:   0%|          |

""


In [8]:
pd.read_gbq("SELECT * FROM richemont.rfm").to_csv("../data/rfm.csv")